## Synergos (FEMNIST dataset)

This notebook serves as a guide to perform an entire federated cycle in three different phases with two worker nodes.

**PHASE 1**. Connect \
**PHASE 2**. Train \
**PHASE 3**. Evaluate

Ensure the TTP, worker_1 and worker_2 containers are already running, otherwise run the following node in sequential.

1. **Run worker_1 node**:

`$ pysyft_worker > docker run -v path_to_femnist_dataset/femnist/data1:/worker/data -v 
path_to_outputs/outputs_1:/worker/outputs --name worker_1 worker:pysyft_demo`

2. **Run worker_2 node**:

`$ pysyft_worker > docker run -v path_to_femnist_dataset/femnist/data1:/worker/data -v path_to_outputs/outputs_2:/worker/outputs --name worker_2 worker:pysyft_demo`

3. **Run TTP node**:

`$ pysyft_ttp > docker run -p 5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v path_to_ttp_data/ttp_data:/ttp/data -v path_to_ml_flow_directory/mlflow_test:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo`


## Initializing Synergos driver

In [1]:
from synergos import Driver

host = "0.0.0.0"
port = 5000

driver = Driver(host=host, port=port)

## Phase 1: CONNECT - Submitting TTP & Participant metadata #

#### 1A. TTP controller creates a project

Create a project which is a classification task with action="classify", if the project is a regression task thenn action="regression".

In [2]:
# 1A. TTP controller creates a project
driver.projects.create(
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': []
    }
)

{'data': {'doc_id': '1',
  'kind': 'Project',
  'key': {'project_id': 'test_project'},
  'relations': {'Experiment': [{'doc_id': '1',
     'kind': 'Experiment',
     'key': {'project_id': 'test_project', 'expt_id': 'test_experiment'},
     'model': [{'is_input': True,
       'structure': {'in_channels': 1,
        'kernel_size': 3,
        'out_channels': 4,
        'padding': 1,
        'stride': 1},
       'l_type': 'Conv2d',
       'activation': 'relu'},
      {'is_input': False, 'l_type': 'Flatten'},
      {'is_input': False,
       'structure': {'bias': True, 'in_features': 3136, 'out_features': 1},
       'l_type': 'Linear',
       'activation': 'softmax'}]}],
   'Run': [{'doc_id': '1',
     'kind': 'Run',
     'key': {'project_id': 'test_project',
      'expt_id': 'test_experiment',
      'run_id': 'test_run'},
     'rounds': 2,
     'epochs': 1,
     'lr': 0.001,
     'lr_decay': 0.1,
     'weight_decay': 0.0,
     'seed': 42,
     'precision_fractional': 5,
     'mu': 0.1,
   

#### 1B. TTP controller creates an experiment

In [3]:
# 1B. TTP controller creates an experiment
driver.experiments.create(
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": "relu",
            "is_input": True,
            "l_type": "Conv2d",
            "structure": {
                "in_channels": 1, 
                "out_channels": 4, # [N, 4, 28, 28]
                "kernel_size": 3,
                "stride": 1,
                "padding": 1
            }
        },
        {
            "activation": None,
            "is_input": False,
            "l_type": "Flatten",
            "structure": {}
        },
        # ------------------------------
        {
            "activation": "softmax",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 4 * 28 * 28,
                "out_features": 1
            }
        }

    ]
)

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'project_id': 'test_project'},
 'data': {'created_at': '2020-09-23 11:15:04 N',
  'key': {'expt_id': 'test_experiment', 'project_id': 'test_project'},
  'model': [{'activation': 'relu',
    'is_input': True,
    'l_type': 'Conv2d',
    'structure': {'in_channels': 1,
     'kernel_size': 3,
     'out_channels': 4,
     'padding': 1,
     'stride': 1}},
   {'activation': None,
    'is_input': False,
    'l_type': 'Flatten',
    'structure': {}},
   {'activation': 'softmax',
    'is_input': False,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 3136, 'out_features': 1}}],
  'relations': {'Run': [{'algorithm': 'FedProx',
     'base_lr': 0.0005,
     'created_at': '2020-09-23 11:06:28 N',
     'criterion': 'NLLLoss',
     'delta': 0.0,
     'epochs': 1,
     'is_snn': False,
     'key': {'expt_id': 'test_experiment',
      'project_id': 'test_project',
      'run_id': 'te

#### 1C. TTP controller creates a run

In [4]:
driver.runs.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

{'data': {'doc_id': '1',
  'kind': 'Run',
  'key': {'project_id': 'test_project',
   'expt_id': 'test_experiment',
   'run_id': 'test_run'},
  'relations': {'Model': [], 'Validation': [], 'Prediction': []},
  'rounds': 2,
  'epochs': 1,
  'lr': 0.001,
  'lr_decay': 0.1,
  'weight_decay': 0.0,
  'seed': 42,
  'precision_fractional': 5,
  'mu': 0.1,
  'l1_lambda': 0.0,
  'l2_lambda': 0.0,
  'base_lr': 0.0005,
  'max_lr': 0.005,
  'patience': 10,
  'delta': 0.0},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'runs.post',
 'params': {'project_id': 'test_project', 'expt_id': 'test_experiment'}}

#### 1D. Participants registers their servers' configurations

In [5]:
driver.participants.create(
    participant_id="test_participant_1",
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [{'doc_id': '2',
     'kind': 'Registration',
     'key': {'project_id': 'test_project',
      'participant_id': 'test_participant_2'},
     'role': 'host'}],
   'Tag': [{'doc_id': '2',
     'kind': 'Tag',
     'key': {'project_id': 'test_project',
      'participant_id': 'test_participant_2'},
     'train': [['train']],
     'evaluate': [],
     'predict': []}]},
  'id': 'test_participant_2',
  'host': '172.17.0.3',
  'port': 8020,
  'log_msgs': True,
  'verbose': True,
  'f_port': 5000},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

#### 1E. Participants registers their role in a specific project

In [6]:
driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="test_project",
    participant_id="test_participant_2",
    role="host"
)

{'data': {'doc_id': '2',
  'kind': 'Registration',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'project': {'incentives': {'tier_2': [], 'tier_1': []}, 'start_at': None},
  'participant': {'id': 'test_participant_2',
   'host': '172.17.0.3',
   'port': 8020,
   'f_port': 5000,
   'log_msgs': True,
   'verbose': True},
  'role': 'host'},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {}}

#### 1F. Participants registers their tags for a specific project

In [7]:
driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_1",
    train=[
        ["train"]
    ],
    evaluate = [["evaluate"]]
)

driver.tags.create(
    project_id="test_project",
    participant_id="test_participant_2",
    train=[["train"]]
)

{'data': {'doc_id': '2',
  'kind': 'Tag',
  'key': {'project_id': 'test_project',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [],
  'predict': []},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'test_project',
  'participant_id': 'test_participant_2'}}

## Phase 2: TRAIN - Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [8]:
driver.alignments.create(project_id="test_project")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'test_project'},
 'data': [{'created_at': '2020-09-23 11:16:28 N',
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'test_project'},
   'link': {'alignment_id': '39dd610afd8e11eaadfd0242ac110004',
    'registration_id': '08c238f2fd8e11ea8b4b0242ac110004',
    'tag_id': '09170a3afd8e11eab1a10242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 1,
   'kind': 'Alignment'},
  {'created_at': '2020-09-23 11:16:28 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project'},
   'link': {'alignment_id': '39e1b8ccfd8e11eaadfd0242ac110004',
    'registration_id': '08b915d8fd8e11eab2620242ac110004',
    'tag_id': '090f993afd8e11ea94c20242ac110004'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 2,
   'kind': 'Alignment'}]}

#### 2B. Trigger training across the federated grid

In [9]:
model_resp = driver.models.create(
        project_id="test_project",
        expt_id="test_experiment",
        run_id="test_run")

## Phase 3: EVALUATE - Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [11]:
driver.validations.create(
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Validation',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'evaluate': {'statistics': {'accuracy': [0.6629213483146067],
     'roc_auc_score': [0.5],
     'pr_auc_score': [0.6685393258426966],
     'f_score': [0.0],
     'TPRs': [0.0, 1.0, 0.0],
     'TNRs': [1.0, 0.0, 1.0],
     'PPVs': [0.0, 0.3258426966292135, 0.0],
     'NPVs': [0.6629213483146067, 0.0, 0.6629213483146067],
     'FPRs': [0.0, 1.0, 0.0],
     'FNRs': [1.0, 0.0, 1.0],
     'FDRs': [0.0, 0.6741573033707865, 0.0],
     'TPs': [0, 29, 0],
     'TNs': [59, 0, 59],
     'FPs': [0, 60, 0],
     'FNs': [30, 0, 30]},
    'res_path': '/worker/outputs/test_project/test_experiment/test_run/evaluate/inference_statistics_evaluate.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'validations.post',
 'params': {'project_id': 'test_project',
  'expt_id': 'test_experim

#### 3B. Perform prediction(s) of combination(s)

In [12]:
driver.predictions.create(
    tags={"test_project": [["predict"]]},
    participant_id="test_participant_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'test_project',
    'expt_id': 'test_experiment',
    'run_id': 'test_run'},
   'predict': {'statistics': {'accuracy': [0.29931972789115646],
     'roc_auc_score': [0.5],
     'pr_auc_score': [0.8503401360544218],
     'f_score': [0.0],
     'TPRs': [0.0, 1.0, 0.0],
     'TNRs': [1.0, 0.0, 1.0],
     'PPVs': [0.0, 0.2108843537414966, 0.0],
     'NPVs': [0.29931972789115646, 0.0, 0.9115646258503401],
     'FPRs': [0.0, 1.0, 0.0],
     'FNRs': [1.0, 0.0, 1.0],
     'FDRs': [0.0, 0.7891156462585034, 0.0],
     'TPs': [0, 62, 0],
     'TNs': [88, 0, 268],
     'FPs': [0, 232, 0],
     'FNs': [206, 0, 26]},
    'res_path': '/worker/outputs/test_project/test_experiment/test_run/predict/inference_statistics_predict.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'predictions.post',
 'params': {'project_id': 'test_project',
  'expt_id': 'test_exper